<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Desafío 2

# ¡Listo para corregir!


## Alumno
Denardi, Fabricio

## Cohorte
15-2024

# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
def get_recipes_files(folder_paths):
    files = []
    for folder_path in folder_paths:
        files.extend([os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.txt') or file.endswith('.md')])
    return files

In [3]:
folder_paths = ['recipes_1', 'recipes_2']
all_files = get_recipes_files(folder_paths)
print("All files:")
print(all_files)

All files:
['recipes_1/almonds.txt', 'recipes_1/fava.txt', 'recipes_1/squares.txt', 'recipes_1/recipes2.txt', 'recipes_1/brownies.txt', 'recipes_1/pannacotta.txt', 'recipes_1/bruschetta.txt', 'recipes_1/sweet_potato_pie.txt', 'recipes_1/sorbet.txt', 'recipes_1/cauli.txt', 'recipes_1/tart.txt', 'recipes_1/cornbread.txt', 'recipes_2/crispy-beef-with-egg-fried-rice.md', 'recipes_2/classic-duck-breast.md', 'recipes_2/crispy-sesame-chicken.md', 'recipes_2/tomato-pasta.md', 'recipes_2/beef-stroganoff.md', 'recipes_2/carrot-cake.md', 'recipes_2/cacio-e-peppe.md', 'recipes_2/pizza-sauce.md', 'recipes_2/tagliatelle-with-broccoli-cauliflower-and-blue-cheese.md', 'recipes_2/pizza-dogs.md', 'recipes_2/meatballs.md', 'recipes_2/chicken-balmoral.md', 'recipes_2/chicken-broth-soup.md', 'recipes_2/thai-green-curry.md', 'recipes_2/blueberry-and-raspberry-muffins.md', 'recipes_2/prawn-pad-thai.md', 'recipes_2/toad-in-the-hole.md', 'recipes_2/garlic-dough-balls.md', 'recipes_2/school-fudge-tart.md', 'rec

In [4]:
# El encoding latin-1 es necesario para leer los archivos de texto. Si no se especifica, se produce un error de UnicodeDecodeError:
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 2: invalid continuation byte
df = pd.DataFrame({'recipe': [open(file, encoding='latin-1').read() for file in all_files]})


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 80


In [6]:
df

,recipe
0,"BLANCHING NUTS: In the case of nuts, especiall..."
1,(fava beans)\n\ntreat gently when young and fr...
2,"These are kid tested, mother approved. From Co..."
3,\nless.\n\n1 cup dried split peas 1/2 teaspoon...
4,Fooder's Brownies (I usually 1/2 the recipe to...
...,...
75,---\ntitle: Beef and Guinness stew\ndate: 2021...
76,---\ntitle: Margherita pizza\ndate: 2021-07-18...
77,---\ntitle: Whisky Haggis sauce\ndate: 2023-01...
78,---\ntitle: Bosnian Stuffed Peppers (Punjene P...


### 1 - Preprocesamiento

In [7]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row.iloc[0]))

    # El iloc reemplazó a la siguiente funcionalidad deprecated: row[0]


In [8]:
# Demos un vistazo
sentence_tokens[:5]

[['blanching',
  'nuts',
  'in',
  'the',
  'case',
  'of',
  'nuts',
  'especially',
  'with',
  'almonds',
  'pistachios',
  'and',
  'hazelnuts',
  'in',
  'addition',
  'to',
  'the',
  'tough',
  'outer',
  'husk',
  'they',
  'have',
  'a',
  'thin',
  'inner',
  'lining',
  'which',
  'needs',
  'to',
  'be',
  'removed',
  'this',
  'lining',
  'can',
  'be',
  'bitter',
  'and',
  'somewhat',
  'unattractive',
  'when',
  'found',
  'in',
  'confection',
  'and',
  'baked',
  'goods',
  'blanching',
  'can',
  'be',
  'a',
  'time',
  'consuming',
  'process',
  'think',
  'about',
  'purchasing',
  'them',
  'already',
  'blanched',
  'and',
  'skinned',
  'nuts',
  'can',
  'then',
  'be',
  'chopped',
  'and',
  'then',
  'toasted',
  'if',
  'desired',
  'almonds',
  'shell',
  'almonds',
  'and',
  'place',
  'them',
  'in',
  'a',
  'saucepan',
  'cover',
  'them',
  'with',
  'water',
  'boil',
  'for',
  '2',
  'to',
  '3',
  'minutes',
  'drain',
  'and',
  'rinse',
 

### 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=10,       # dimensionalidad de los vectores 
                     negative=5,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [11]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 80


In [13]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 978


### 3 - Entrenar embeddings

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 264337.40625
Loss after epoch 1: 195538.3125
Loss after epoch 2: 181358.46875
Loss after epoch 3: 172058.5625
Loss after epoch 4: 164563.6875
Loss after epoch 5: 160262.4375
Loss after epoch 6: 156458.25
Loss after epoch 7: 155151.5
Loss after epoch 8: 151493.75
Loss after epoch 9: 150952.125
Loss after epoch 10: 150243.5
Loss after epoch 11: 149340.125
Loss after epoch 12: 144449.375
Loss after epoch 13: 144011.25
Loss after epoch 14: 143827.0
Loss after epoch 15: 143048.0
Loss after epoch 16: 143326.75
Loss after epoch 17: 141953.25
Loss after epoch 18: 141581.0
Loss after epoch 19: 141333.5
Loss after epoch 20: 141420.75
Loss after epoch 21: 141801.0
Loss after epoch 22: 140593.25
Loss after epoch 23: 141539.75
Loss after epoch 24: 141755.5
Loss after epoch 25: 140899.0
Loss after epoch 26: 140390.25
Loss after epoch 27: 130377.75
Loss after epoch 28: 130332.0
Loss after epoch 29: 129430.0
Loss after epoch 30: 129419.0
Loss after epoch 31: 128385.5
Loss after epo

(4930191, 7573600)

### 4 - Ensayar

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["chocolate"], topn=10)

[('chips', 0.9587956666946411),
 ('caramel', 0.8662093877792358),
 ('unsweetened', 0.8509345054626465),
 ('piece', 0.8420158624649048),
 ('substitute', 0.8306741118431091),
 ('sweetened', 0.8303201198577881),
 ('heavy', 0.8193446397781372),
 ('topping', 0.8093628883361816),
 ('bitter', 0.8066312074661255),
 ('pre', 0.7897059917449951)]

#### Comentarios
En esta primer palabra elegida para analizar, vemos claramente la eficacia del modelo. Choclate chips es una preparación de excelencia que encontraremos en cualquier set de recetas de pastelería.

Luego vemos, diferentes elemmentos que suelen combinarse con el chocolate, como la banana, el caramelo. Y en otras el tipo de chocolate, como el amargo (unsweetened).  También encontramos la relación con preparaciones menos convencionales, como el cocktail, el maiz (corn).

Veremos ahora otros análisis, pero anticipo un buen rendimiento del modelo.

In [16]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["pepper"], topn=10)

[('refrigerated', 0.25176477432250977),
 ('16', 0.23496967554092407),
 ('yield', 0.20719610154628754),
 ('cutting', 0.2057875692844391),
 ('makes', 0.19414596259593964),
 ('note', 0.1866300106048584),
 ('approx', 0.1845967024564743),
 ('24', 0.1758279651403427),
 ('brownies', 0.1497783660888672),
 ('these', 0.13892851769924164)]

#### Comentarios
Muchas de estos términos con similaridad más ortogonal, son núneros, con lo cual tienen que ver con el corpus elegido. Lo interesante es que brownies o pan de maiz, son algo que dificilmente lleve pimienta, y uno no corta (cutting) granos de pimienta, a lo sumo los muele, pero no con cuchillo.

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["carrot"], topn=10)

[('mums', 0.9339948892593384),
 ('cake', 0.8978996276855469),
 ('blueberry', 0.8737064599990845),
 ('bosnian', 0.8513693809509277),
 ('soup', 0.8412330746650696),
 ('classic', 0.8380641937255859),
 ('easy', 0.8354212045669556),
 ('tart', 0.8335493206977844),
 ('ð\x9f¥³', 0.8317363858222961),
 ('james', 0.8286657929420471)]

#### Comentarios
En este caso, vemos preparaciones realizadas con zanahoria, com por ejemplo cake/pie/tart (torta), soup (sopa), gudge (ganache, que es una especie de glaseado), el resto las asocio al contexto del set, ya que no están estrechamente relacionadas con zanahora.

Aclaración imporante: cuando digo que está relacionado con el contexto, me refiero a que si agrego otros dataset de diferentes origenes, posiblemente no obtenga estas palabras, quiero decir que es poco probable que aparezcan en otras recetas. En cambio carrot cake, es algo que en cualquier libro de pastelería veremos.

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["oven"], topn=5)

[('degrees', 0.9188392758369446),
 ('350', 0.9182568788528442),
 ('preheat', 0.9119279980659485),
 ('line', 0.8807625770568848),
 ('middle', 0.8748724460601807)]

### Comentarios
Creo que acá es donde vemos las mejores analogías. 350 son los grados en Farenheit, que equivalen a 180 centígrados, temperatura más usada en el horneado de alimentos.

Luego vemos palabras como preheat, que significa precalentar el horno, usado usualmente en el preparado de tortas, budines, panes. Degrees que es la unidad de medida de temperatura (Grados). Middle, que es precalentar a temperatura media (muy usado en pastelería).

El match con line lo asocio exclusivamente con el corpus elegido, ya que no es un término habitual en la gastronomía.

In [19]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["knife"], topn=5)

[('bã©chamel', 0.20377616584300995),
 ('easy', 0.17771506309509277),
 ('honey', 0.17690801620483398),
 ('pad', 0.16657023131847382),
 ('oyster', 0.16407357156276703)]

#### Comentarios
Claramente la primer relación negativa la vemos en que la miel, en el contexto de preparados de recetas, no se suele servir con cuchillos. El resto de las relaciones, son palabras que nada tiene que ver con el contexto gastronómico (ej: png es un formato de imagen).

In [20]:
# el método `get_vector` permite obtener los vectores:
vector_chocolate = w2v_model.wv.get_vector("chocolate")
print(vector_chocolate)

[ 1.1884283   0.6608797   0.09293447  1.3739711   0.6037429   0.13895622
 -0.6739304   0.97853637 -0.39636517 -0.7122866 ]


In [21]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_chocolate)

[('chocolate', 0.9999998807907104),
 ('chips', 0.9587956666946411),
 ('caramel', 0.8662093877792358),
 ('unsweetened', 0.8509345054626465),
 ('piece', 0.8420158624649048),
 ('substitute', 0.8306741118431091),
 ('sweetened', 0.8303201198577881),
 ('heavy', 0.8193446397781372),
 ('topping', 0.8093628883361816),
 ('bitter', 0.8066312074661255)]

#### Comentarios
Ya analizamos estos valores anteriormente.

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["rice"], topn=10)

[('ratatouille', 0.8935331106185913),
 ('fried', 0.8907067179679871),
 ('saffron', 0.8533592224121094),
 ('broccoli', 0.8524351119995117),
 ('100g', 0.851051926612854),
 ('meat', 0.8482113480567932),
 ('florets', 0.8376943469047546),
 ('cauliflower', 0.8206608295440674),
 ('title', 0.8188447952270508),
 ('cooked', 0.8066512942314148)]

#### Comentarios
Dado que el source contiene muchas recetas indias, el primer match se da con el cordero, que es un plato típico de ese país. Adicionalmente, vemos que las siguientes coincidencia la obtenemos o con sustitutos, uso del arroz comom acompañamiento del plato principal, o combinación de ingredientes junto con el arroz.

In [23]:
test_vector = w2v_model.wv["vinegar"].shape
test_vector

(10,)

### Test de analogía por resta de vectores
Simplemente para ampliar mi análisis, decidí construir un test de analogía como tal, restando vestores y buscando la similaridad del coseno entre ambos.

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

def test_analogy(model: Word2Vec, word1: str, word2: str, word3: str, word4: str) -> float:
    try:
        vector1 = model.wv[word1]
        vector2 = model.wv[word2]
        vector3 = model.wv[word3]
        vector4 = model.wv[word4]

        result = cosine_similarity([vector2 - vector1], [vector4 - vector3])[0][0]
        return result
    except KeyError:
        return 0
    


In [25]:
analogy_list = [
    ("banana", "fruit", "carrot", "vegetable"),
    ("teaspoon", "salt", "cup", "sugar"),
    ("salt", "pepper", "sugar", "chocolate"),
    ("flour", "bake", "water", "boil"),
    ("bread", "butter", "pasta", "sauce"),
    ("onion", "soup", "garlic", "pasta"),
]

for analogy in analogy_list:
    result = test_analogy(w2v_model, analogy[0], analogy[1], analogy[2], analogy[3])

    print(f'Analogía: "{analogy[0]}" es a "{analogy[1]}" como "{analogy[2]}" es a "{analogy[3]}"')
    print(f"Similaridad: {result:.2f}")
    print("-----------------------------")

Analogía: "banana" es a "fruit" como "carrot" es a "vegetable"
Similaridad: 0.36
-----------------------------
Analogía: "teaspoon" es a "salt" como "cup" es a "sugar"
Similaridad: 0.22
-----------------------------
Analogía: "salt" es a "pepper" como "sugar" es a "chocolate"
Similaridad: 0.16
-----------------------------
Analogía: "flour" es a "bake" como "water" es a "boil"
Similaridad: 0.56
-----------------------------
Analogía: "bread" es a "butter" como "pasta" es a "sauce"
Similaridad: 0.29
-----------------------------
Analogía: "onion" es a "soup" como "garlic" es a "pasta"
Similaridad: 0.71
-----------------------------


#### Conclusión del punto
He encontrado algunas analogías interesantes. Ahora bien, como era de esperarse, no logré detectar una analogía perfecta. Esto se debe a que dependemos exclusivamente del corpus elegido. Si bien son todas recetas, las mismas perteneces a diferentes orígenes de datos y además son de diferentes paises, culturas, sabores, etc, lo que hace que tenga relaciones muy heterogénas.

### 5 - Visualizar agrupación de vectores

In [26]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [27]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [28]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

In [29]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

In [30]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors_recipes.tsv", vectors, delimiter="\t")

with open("label_recipes.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

# Conclusiones finales
1. La performance del modelo entrenado es muy buena. A pesar de las limitaciones del mismo, pude encontrar relaciones muy interesantes, la mayoría de esas esperadas, aunque otras que dependen exclusivamente del contexto de las recetas elegidas, que requeriran de empliar el dataset para evaluar si siguen o no siendo relevamentes, en cuyo caso, habría que ver el origen de esta relación.

2. Para el análisis de este y cualquier problema, es necesario contar con conocimientos del tema elegido, al menos para evaluar la efectividad del mismo. Mi hobbie en gastronomía, me permitió realizar un buen análisis del mismo.

3. Vemos en el gráfico del punto anterior que la reducción de dimensiones, tuvo bastante éxito. Veo, sin embargo, que palabras genéricas o no asociadas a la gastronomía, tienen cercanía con otros término vinculados a dicha temática, dificultando, a mi criterio, un análisis más limpio. No quise seguir sumando al hiperparámetro de cantidad mínima de apariciones, porque me eliminaba términos muy interesante y que, si bien no aparecen en todas las recetas, están estrechamente relacionadas con el tema de interés, como por ejemplo la canela.

## Próximos pasos
1. Buscar documentos con recetas más variadas y de diferentes origines, país, cultura, para lograr mejorar la peformance del algoritmo.

2. A lo largo de la materia veremos algoritmos y modelso más complejos que nos permitirán encontrar una mejor solución.

